In [49]:
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from urllib.parse import quote
import datetime
import pytz
import subprocess

In [50]:
TITLE = 'Bibiolography of Coconut Rhinoceros Beetle in the Philippines'
CSV_PATH = 'crb-pi-bib.csv'
TEMPLATE_PATH = 'crb-pi-bib-template.html'
CSS_PATH = 'crb-pi-bib.css'
HTML_PATH = 'index.html'

In [51]:
# Read CSV

df = pd.read_csv(CSV_PATH)

# Reorder columns

df = df[['Key','Item Type','Author','Publication Year','Title','Url','File Attachments','Notes', 'Abstract Note', 'Extra']]

# Sort by author and year

df.sort_values(by=['Author','Publication Year'], inplace=True)

# reset index

df = df.reset_index(drop=True)

In [52]:
# create new column which contains a formatted reference

def format_ref(row):

    if pd.isna(row)['Author']:
        author = 'Unknown'
    else:
        author = row['Author']

    if pd.isna(row['Publication Year']):
        year = 0
    else:
        year = row['Publication Year']

    title = row['Title']

    if pd.isna(row['Url']):
        url = '' 
    else:
        url = row['Url']

    return f'{author} {int(year)}<br><b>{title}</b><br><a href="{url}">{url}</a>'

df['ref'] = df.apply(format_ref, axis=1)

In [53]:
# create abstract_exists column

def abstract_exists(row):
    return pd.notna(row['Abstract Note'])
df['abstract_exists'] = df.apply(abstract_exists, axis=1)

# create notes_exist column

def notes_exist(row):
    return pd.notna(row['Notes'])

df['notes_exist'] = df.apply(notes_exist, axis=1)

# create attachments_exist column

def attachments_exist(row):
    return pd.notna(row['File Attachments'])

df['attachments_exist'] = df.apply(attachments_exist, axis=1)

# create a new column for formatted attachment urls

def format_attachments(row):
    if row['attachments_exist']:
    
        s = row['File Attachments']
        urls = s.split(';')

        attachments = ''
        for url in urls:
            s = url.strip() # remove whitespace from ends of string
            s = s.replace('/home/aubrey/snap/zotero-snap/common/Zotero/storage/', '')
            # s = quote(s)    # convert space characters to %20
            # s = f'<a href="{s}">{s}</a><br>'
            attachments += s
    
        return attachments
    else:
        return ''
    
df['attachments'] = df.apply(format_attachments, axis=1)

df


,Key,Item Type,Author,Publication Year,Title,Url,File Attachments,Notes,Abstract Note,Extra,ref,abstract_exists,notes_exist,attachments_exist,attachments
0,999WP46H,thesis,"Adams, Brandi-Leigh H",2019.0,ANALYSIS AND DEVELOPMENT OF MANAGEMENT TOOLS F...,https://scholarspace.manoa.hawaii.edu/bitstrea...,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,NaN,Citation Key: adams_analysis_2019,"Adams, Brandi-Leigh H 2019<br><b>ANALYSIS AND ...",False,False,True,G65232JM/Adams et al. - TROPICAL PLANT PATHOLO...
1,WMAEHGUH,bookSection,"Ali Ahmed, Abdelhameed Ibrahim; Sherif Hussein",2019.0,Detection of Palm Tree Pests Using Thermal Ima...,NaN,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,Due to the extensive stretches of date plantat...,Citation Key: ali_ahmed_detection_2019,"Ali Ahmed, Abdelhameed Ibrahim; Sherif Hussein...",True,False,True,7Z9BAPX8/10.1007978-3-030-02357-712.pdf
2,PKV4T89V,bookSection,"Ali, Siti Ramlah Ahmad; Moslim, Ramle; Kamarud...",2010.0,MICROBIAL APPROACH IN PEST CONTROL,NaN,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,NaN,Citation Key: ali_microbial_nodate,"Ali, Siti Ramlah Ahmad; Moslim, Ramle; Kamarud...",False,False,True,27B5P58F/Ali et al. - MICROBIAL APPROACH IN PE...
3,TMS79AFM,journalArticle,"Anggraini, Erise; Sinaga, Tessia; Irsan, Chand...",2024.0,Insects found in different ages of coconut vir...,NaN,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,"Anggraini E, Sinaga TM, Irsan C, Herlinda S, M...",NaN,"Anggraini, Erise; Sinaga, Tessia; Irsan, Chand...",True,False,True,IS24JH74/Anggraini et al. - 2024 - Insects fou...
4,B7IX8S3N,conferencePaper,Anonymous,2022.0,Coconut rhinoceros beetle (CRB) strategic acti...,NaN,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,NaN,NaN,Anonymous 2022<br><b>Coconut rhinoceros beetle...,False,False,True,DH9J4UVR/SAP CRB - Draft 16-09-2022.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,IQAGQNVJ,journalArticle,"Zelazny, Bernhard",1977.0,Occurrence of the baculovirus disease of the c...,NaN,NaN,NaN,Examination of populations of the coconut pest...,Citation Key: zelazny_b._occurrence_1977,"Zelazny, Bernhard 1977<br><b>Occurrence of the...",True,False,False,
128,I2Z3UZUK,webpage,NaN,2023.0,DOST-PCAARRD launches 2 projects to assist far...,https://mb.com.ph/2023/01/18/dost-pcaarrd-laun...,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,The Nation's Leading Newspaper,Section: National,Unknown 2023<br><b>DOST-PCAARRD launches 2 pro...,True,False,True,AZTQIV6S/dost-pcaarrd-launches-2-projects-to-a...
129,JQ9DF94U,webpage,NaN,NaN,Green waste from Typhoon Dolphin to be dispose...,http://www.postguam.com/news/local/green-waste...,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,NaN,Citation Key: noauthor_green_nodate,Unknown 0<br><b>Green waste from Typhoon Dolph...,False,False,True,4UJP6ZX5/article_13850354-ce28-11e5-849f-e33b6...
130,4DQNN5YJ,blogPost,NaN,NaN,Dead coconut rhinoceros beetle found on Maui |...,https://mauinow.com/2023/09/14/dead-coconut-rh...,/home/aubrey/snap/zotero-snap/common/Zotero/st...,NaN,One dead coconut rhinoceros beetle was found o...,NaN,Unknown 0<br><b>Dead coconut rhinoceros beetle...,True,False,True,8CGYSD6Y/dead-coconut-rhinoceros-beetle-found-...


In [54]:
def get_timestamp():
    return datetime.datetime.now(pytz.timezone('Asia/Manila')).isoformat('T', 'minutes')

# get_timestamp()

In [55]:
def get_preamble_html():
    with open('preamble.html') as f:
        preamble_html = f.read()
    return preamble_html

In [56]:
templateLoader = FileSystemLoader(searchpath=".")
templateEnv = Environment(loader=templateLoader)
template = templateEnv.get_template(TEMPLATE_PATH)
header = f'{TITLE}<br>by Aubrey Moore | {get_timestamp()}'
templateVars = {'df': df, 'header': header, 'preamble_html': get_preamble_html()}
outputText = template.render(templateVars)

with open(HTML_PATH, 'w') as f:
    f.write(outputText)
print(f'The bibliography is saved in {HTML_PATH}.')
print('Finished')

# subprocess.run(['firefox', 'index.html'])

The bibliography is saved in index.html.
Finished
